![facs2/4](https://img.shields.io/badge/facs2/4-lightgrey)
[![Jupyter Notebook](https://img.shields.io/badge/Source%20on%20GitHub-orange)](https://github.com/laminlabs/lamin-usecases/blob/main/docs/facs2.ipynb)
[![lamindata](https://img.shields.io/badge/Source%20%26%20report%20on%20LaminHub-mediumseagreen)](https://lamin.ai/laminlabs/lamindata/record/core/Transform?uid=SmQmhrhigFPLz8)

# Append a new shard of data

We have one artifact in storage and are about to receive a new shard of data.

In this notebook, we'll see how to manage the situation.

In [ ]:
import lamindb as ln
import bionty as bt
import readfcs

bt.settings.organism = "human"

In [ ]:
ln.transform.stem_uid = "SmQmhrhigFPL"
ln.transform.version = "0"
ln.track()

## Ingest a new artifact

### Access ![](https://img.shields.io/badge/Access-10b981)

Let us validate and register another `.fcs` file from [Oetjen18](https://insight.jci.org/articles/view/124928):

In [ ]:
filepath = readfcs.datasets.Oetjen18_t1()

adata = readfcs.read(filepath)

In [ ]:
adata

## Transform: normalize ![](https://img.shields.io/badge/Transform-10b981)

In [ ]:
import anndata as ad
import pytometry as pm

In [ ]:
pm.pp.split_signal(adata, var_key="channel")

In [ ]:
pm.pp.compensate(adata)

In [ ]:
pm.tl.normalize_biExp(adata)

In [ ]:
adata = adata[  # subset to rows that do not have nan values
    adata.to_df().isna().sum(axis=1) == 0
]

In [ ]:
adata.to_df().describe()

### Validate cell markers ![](https://img.shields.io/badge/Validate-10b981) 

Let's see how many markers validate:

In [ ]:
validated = bt.CellMarker.validate(adata.var.index)

Let's standardize and re-validate:

In [ ]:
adata.var.index = bt.CellMarker.standardize(adata.var.index)
validated = bt.CellMarker.validate(adata.var.index)

Next, register non-validated markers from Bionty:

In [ ]:
records = bt.CellMarker.from_values(adata.var.index[~validated])
ln.save(records)

Manually create 1 marker:

In [ ]:
bt.CellMarker(name="CD14/19").save()

Move metadata to obs:

In [ ]:
validated = bt.CellMarker.validate(adata.var.index)
adata.obs = adata[:, ~validated].to_df()
adata = adata[:, validated].copy()

Now all markers pass validation:

In [ ]:
validated = bt.CellMarker.validate(adata.var.index)
assert all(validated)

### Register ![](https://img.shields.io/badge/Register-10b981) 

In [ ]:
features = ln.Feature.lookup()
efs = bt.ExperimentalFactor.lookup()
organism = bt.Organism.lookup()
markers = bt.CellMarker.lookup()

In [ ]:
artifact = ln.Artifact.from_anndata(
    adata,
    description="Oetjen18_t1"
)

In [ ]:
artifact.save()

In [ ]:
artifact.features.add_from_anndata(field=bt.CellMarker.name)

In [ ]:
artifact.labels.add(efs.fluorescence_activated_cell_sorting, features.assay)
artifact.labels.add(organism.human, features.organism)

In [ ]:
artifact.features

View data flow:

In [ ]:
artifact.view_lineage()

Inspect a PCA fo QC - this collection looks much like noise:

In [ ]:
import scanpy as sc

sc.pp.pca(adata)
sc.pl.pca(adata, color=markers.cd8.name)

## Create a new version of the collection by appending a artifact

Query the old version:

In [ ]:
collection_v1 = ln.Collection.filter(name="My versioned cytometry collection").one()

In [ ]:
collection_v2 = ln.Collection(
    [artifact, collection_v1.artifact], is_new_version_of=collection_v1, version="2"
)

In [ ]:
collection_v2

In [ ]:
collection_v2.features

In [ ]:
collection_v2

In [ ]:
collection_v2.save()

In [ ]:
collection_v2.labels.add(efs.fluorescence_activated_cell_sorting, features.assay)
collection_v2.labels.add(organism.human, features.organism)

In [ ]:
collection_v2.view_lineage()